In [1]:
import pyphi
from visualize_pyphi import *
from visualize_pyphi import utils
import pickle as pkl
from tqdm.auto import tqdm
from pyphi.models.subsystem import FlatCauseEffectStructure as sep
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [4]:
'''
Check that the config parameters are correct (if not change pyphi_config.yml to match):
REPERTOIRE_DISTANCE = 'IIT_4.0_SMALL_PHI'
IIT_VERSION = 'maximal-state-first'
PARTITION_TYPE = 'TRI'
'''
pyphi.config.REPERTOIRE_DISTANCE,pyphi.config.IIT_VERSION,pyphi.config.PARTITION_TYPE

('IIT_4.0_SMALL_PHI', 'maximal-state-first', 'TRI')

In [3]:
from graphiit import Graph
import pyphi
import sys
import numpy


pyphi.config.PARALLEL_CONCEPT_EVALUATION = True
#pyphi.config.NUMBER_OF_CORES = -1
#pyphi.config.MAXIMUM_CACHE_MEMORY_PERCENTAGE = 100

graph_config = [
('x1','AND','x3','x4'),
('x3','XNOR','x1','x4','x5'),
('x4','NAND','x1','x3','x5','x9'),
('x5','XOR','x4','x7','x9'),
('x7','OR','x5','x9'),
('x9','XNOR','x4','x5','x7'),
]

graph = Graph(graph_config)

network = graph.pyphi_network()

state = (0, 1, 1, 0, 0, 1)

subsystem = pyphi.Subsystem(network, state)

x1, x3, x4, x5, x7, x9 = subsystem.node_indices

In [5]:
#Compute all distinctions
distinctions = pyphi.compute.ces(subsystem)

In [6]:
# # Print distinctions (optional)
distinctions

═══════════════════════════════════════════════════════════════════════════
                    Cause-effect structure (10 concepts)                   
═══════════════════════════════════════════════════════════════════════════
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
  Concept: mechanism = [x1], state = [0], φ = 0.0931094  
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
         MIC               MIE          
  ┌────────────────┐┌────────────────┐  
  │  φ = 0.276692  ││  φ = 0.0931094 │  
  │  Purview: [x3] ││  Purview: [x4] │  
  └────────────────┘└────────────────┘  
    
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
  Concept: mechanism = [x3], state = [1], φ = 1/4  
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
            MIC                  MIE          
  ┌──────────────────────┐┌────────────────┐  
  │  φ = 1/4             ││  φ = 1/2       │  
  │  Purview: [x1,x4,x5] ││  Purview: [x1] │  
  └──────────────────────┘└────────────────┘  
    
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
#Compute all relations
relations = list(pyphi.relations.relations(subsystem,sep(distinctions),max_degree=2))

  0%|          | 0/137 [00:00<?, ?it/s]

/home/diego/anaconda3/envs/IIT/lib/python3.9/site-packages/pyphi/utils.py:87: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON


In [8]:
# # Print relations (optional)
relations

[Relation(relata=Relata({Cause[5]/[2,3,4], Cause[1]/[0,2,3]}), purview=frozenset({2, 3}), phi=0.25),
 Relation(relata=Relata({Cause[1]/[0,2,3], Effect[1]/[0]}), purview=frozenset({0}), phi=0.25),
 Relation(relata=Relata({Cause[1,2]/[0,2,3], Effect[1,5]/[2,4]}), purview=frozenset({2}), phi=0.19999999999999998),
 Relation(relata=Relata({Cause[5]/[2,3,4], Effect[3]/[2,4]}), purview=frozenset({2, 4}), phi=0.25),
 Relation(relata=Relata({Cause[0,2,3]/[2,4,5], Effect[3]/[2,4]}), purview=frozenset({4}), phi=0.35555555555555557),
 Relation(relata=Relata({Cause[1]/[0,2,3], Effect[1,2]/[0]}), purview=frozenset({0}), phi=0.25),
 Relation(relata=Relata({Effect[1,2,5]/[0,2], Cause[3]/[2,4,5]}), purview=frozenset({2}), phi=0.25),
 Relation(relata=Relata({Effect[1]/[0], Cause[2]/[0,1]}), purview=frozenset({0}), phi=0.044478484267289564),
 Relation(relata=Relata({Cause[1,2,5]/[0,2,3,4], Effect[1,2,5]/[0,2]}), purview=frozenset({2}), phi=0.39999999999999997),
 Relation(relata=Relata({Effect[5]/[4], Eff

In [9]:
ces = pyphi.big_phi.sia(subsystem,distinctions,relations)

Returning trivially-reducible SIA


In [10]:
max_distintions = ces.phi_structure.distinctions
max_relations = ces.phi_structure.relations

In [11]:
#Plot the cause-effect structure
fig = visualize_ces.plot_ces(
    subsystem,
    sep(max_distintions),
    max_relations,
    network_name="example_ces",
)

fig.show()

IndexError: list index out of range

In [12]:
# Print distinctions
utils.sepces2df(sep(max_distintions),subsystem)

,mechanism,direction,purview,state,phi
0,x1,CAUSE,x3,0,0.276692
1,x1,EFFECT,x4,1,0.093109
2,x3,CAUSE,x1x4x5,000\n110\n101\n011,0.250000
3,x3,EFFECT,x1,1,0.500000
4,x4,CAUSE,x1x3,11,0.064386
5,x4,EFFECT,x1,1,0.500000
6,x5,CAUSE,x4x7x9,000\n110\n101\n011,0.250000
7,x5,EFFECT,x4x7,10,0.546555
8,x9,CAUSE,x4x5x7,000\n110\n101\n011,0.250000
9,x9,EFFECT,x7,1,0.415037


In [13]:
sorted(max_relations,key=len)

[Relation(relata=Relata({Effect[0,1,2,4]/[0,1,2,3], Cause[2]/[0,1]}), purview=frozenset({0}), phi=0.044478484267289564),
 Relation(relata=Relata({Cause[4]/[2,3], Effect[0,1,2,4]/[0,1,2,3]}), purview=frozenset({2, 3}), phi=0.5),
 Relation(relata=Relata({Cause[0,1,2,4]/[0,2,3], Cause[0,1,2]/[0,2]}), purview=frozenset({0, 2}), phi=1.0270873349168606),
 Relation(relata=Relata({Cause[0,1,2]/[0,2], Cause[0,2,3]/[1,2,4]}), purview=frozenset({2}), phi=0.5629446374781043),
 Relation(relata=Relata({Cause[0,1,2,4]/[0,2,3], Effect[4]/[2]}), purview=frozenset({2}), phi=0.125),
 Relation(relata=Relata({Effect[0,1,2]/[0,1,2], Effect[0,2,3]/[1,2,4]}), purview=frozenset({1, 2}), phi=1.0931094043914815),
 Relation(relata=Relata({Cause[0,2]/[1], Effect[0,2,3]/[1,2,4]}), purview=frozenset({1}), phi=0.3314351610038171),
 Relation(relata=Relata({Effect[0,1,2]/[0,1,2], Effect[2]/[0]}), purview=frozenset({0}), phi=0.5),
 Relation(relata=Relata({Effect[0,1,2]/[0,1,2], Effect[0,2]/[1,2]}), purview=frozenset({1,